<div style="display: flex; align-items: center; gap: 18px; margin-bottom: 15px;">
  <img src="https://files.codebasics.io/v3/images/sticky-logo.svg" alt="Codebasics Logo" style="display: inline-block;" width="130">
  <h1 style="font-size: 34px; color: #1f4e79; margin: 0; display: inline-block;">Codebasics Practice Room - Data Engineering Bootcamp </h1>
</div>


#### ⏱️ Sliding Window Metrics with Structured Streaming

This notebook demonstrates how to compute a **rolling “last 30 minutes error count”**
from application logs using **Spark Structured Streaming**.

We simulate real-time ingestion by converting an existing large CSV file
into multiple small files and reading them as a **stream**.


## 📂 Dataset

**Dataset Name:** `app_logs_large.csv`  


### Columns:
- `event_time`
- `level`
- `service`
- `message`

> ⚠️ In real production systems, logs arrive continuously via **Kafka**.  
> For learning purposes, we simulate streaming using files.


## 🗂️ Scenario

The business wants a **real-time operational metric**:

> **“How many ERROR logs occurred in the last 30 minutes?”**

Requirements:
- Metric should update continuously
- Must be based on **event time**
- Must handle **late-arriving events**
- Should scale as log volume grows

You are asked to implement this using **Spark Structured Streaming**.

---

## 🎯 Task

1. Read historical log data as batch
2. Convert it into a streaming-friendly format
3. Read logs as a streaming DataFrame
4. Filter ERROR logs
5. Apply a **30-minute sliding window** (slide every 1 minute)
6. Add a watermark for late data
7. Output rolling error counts

---

## 🧩 Assumptions

- Logs are event-time based
- Data may arrive late
- Databricks **Serverless compute** is used
- Unity Catalog storage is available

---

## 📦 Deliverables

- Streaming aggregation computing rolling error counts
- Output grouped by:
  - window
  - service

### Expected Columns

| window.start | window.end | service | error_count |
|--------------|------------|---------|-------------|

---

## 🧠 Notes
- Streaming reads **directories**, not single files
- New files represent new streaming data
- Watermarks control late data handling
- Sliding windows enable rolling metrics



## 🧠 Solution Strategy (High-Level)

1. Read existing logs as a batch DataFrame
2. Split batch data into multiple small files
3. Read those files using `readStream`
4. Filter ERROR events
5. Apply sliding window aggregation
6. Write streaming results using a serverless-safe trigger

Spark handles:
- incremental processing
- stateful window aggregation
- fault tolerance using checkpoints


In [0]:
from pyspark.sql import functions as F


## 1. Read Existing Logs (Batch)


In [0]:
batch_logs_df = (
    spark.read
         .option("header", "true")
         .option("inferSchema", "true")
         .csv("your_data")
         .withColumn("event_time", F.to_timestamp("event_time"))
)

display(batch_logs_df.limit(5))


In [0]:
logs_df.printSchema()


## 2. Convert Batch Data into Streaming Input


In [0]:
(
    batch_logs_df
        .repartition(5)   # simulate data arriving in chunks
        .write
        .mode("overwrite")
        .option("header", "true")
        .csv("your_directory")
)


##3. Read Logs as a Stream


In [0]:
logs_df = (
    spark.readStream
         .schema("event_time STRING, level STRING, service STRING, message STRING")
         .option("header", "true")
         .csv("your_directory")
         .withColumn("event_time", F.to_timestamp("event_time"))
)


## 🚨 Filter ERROR Logs


In [0]:
errors_df = logs_df.filter(F.col("level") == "ERROR")


## ⏱️ Sliding Window Aggregation


In [0]:
windowed_errors = (
    errors_df
        .withWatermark("event_time", "5 minutes")
        .groupBy(
            F.window("event_time", "30 minutes", "1 minute"),
            F.col("service")
        )
        .agg(F.count("*").alias("error_count"))
)


## 📤 Write Streaming Output


In [0]:
query = (
    windowed_errors
        .writeStream
        .outputMode("update")
        .format("console")   # demo output
        .option("truncate", "false")
        .option(
            "checkpointLocation",
            "your_directory"
        )
        .trigger(availableNow=True)   # serverless-safe
        .start()
)


## 🧠 How This Works Internally

- Spark tracks state per (window, service)
- Each event contributes to multiple overlapping windows
- Watermark controls how long Spark waits for late data
- Checkpoints ensure fault tolerance and exactly-once semantics


## ✅ Summary

- Sliding windows enable rolling real-time metrics
- File-based streaming simulates Kafka ingestion
- Watermarks handle late data
- Unity Catalog volumes ensure serverless compatibility

This pattern is **production-grade** and widely used
in monitoring and observability systems.
